In [1]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [2]:
train_path = glob.glob('./鸟类品种识别挑战赛训练集/training_set/*/*')
np.random.shuffle(train_path)

In [3]:
train_path

['./鸟类品种识别挑战赛训练集/training_set/Indian Roller/ML126048471.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Gray Wagtail/ML403566341.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Northern Lapwing/ML513516761.jpg',
 './鸟类品种识别挑战赛训练集/training_set/White Wagtail/ML509386231.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Northern Lapwing/ML205206131.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Indian Pitta/ML77282711.jpg',
 './鸟类品种识别挑战赛训练集/training_set/House Crow/ML398037261.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Red-Wattled Lapwing/ML398283661.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Asian Green Bee-Eater/ML117344941.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Asian Green Bee-Eater/ML226730091.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Indian Roller/ML152090321.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Common Kingfisher/ML253510981.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Gray Wagtail/ML415395171.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Indian Peacock/ML98750141.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Forest Wagtail/ML218388861.jpg',
 './鸟类品种识别挑战赛训练集/training_set/Northern Lapwi

In [4]:
class_names = ['Asian Green Bee-Eater', 'Brown-Headed Barbet', 'Cattle Egret',
       'Common Kingfisher', 'Common Myna', 'Common Rosefinch',
       'Common Tailorbird', 'Coppersmith Barbet', 'Forest Wagtail',
       'Gray Wagtail', 'Hoopoe', 'House Crow', 'Indian Grey Hornbill',
       'Indian Peacock', 'Indian Pitta', 'Indian Roller',
       'Jungle Babbler', 'Northern Lapwing', 'Red-Wattled Lapwing',
       'Ruddy Shelduck', 'Rufous Treepie', 'Sarus Crane', 'White Wagtail',
       'White-Breasted Kingfisher', 'White-Breasted Waterhen']
len(class_names)

25

In [11]:
class XunFeiDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        for idx, label in enumerate(class_names):
            if label in self.img_path[index]:
                label = idx
                break
        return img, torch.from_numpy(np.array(label).astype(int))
    
    def __len__(self):
        return len(self.img_path)

In [12]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet50(True)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    
    def forward(self, img):
        out = self.resnet(img)
        return out

In [13]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Train loss', loss.item())
            
        train_loss += loss.item()
    
    return train_loss/len(train_loader)
            
def validate(val_loader, model, criterion):
    model.eval()
    
    val_acc = 0.0
    
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)

            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            val_acc += (output.argmax(1) == target).sum().item()
            
    return val_acc / len(val_loader.dataset)

def predict(test_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    
    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            target = target.to(device)

            # compute output
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)

In [14]:
train_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[:-500],
    transforms.Compose([
                transforms.Resize(256),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])), batch_size=15, shuffle=True, num_workers=4, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[-500:],
    transforms.Compose([
                transforms.Resize(256),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])), batch_size=30, shuffle=False, num_workers=1, pin_memory=False
)

In [15]:
model = XunFeiNet()
model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.001)

In [16]:
for _  in range(10):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc  = validate(val_loader, model, criterion)
    
    print(train_loss, val_acc)

Train loss 3.407654047012329
Train loss 3.1887123584747314
Train loss 3.050079822540283
Train loss 2.8761777877807617
Train loss 2.5200181007385254
Train loss 2.500518798828125
Train loss 2.4960296154022217
Train loss 2.230742931365967
Train loss 1.722125768661499
Train loss 1.8735716342926025
Train loss 2.0721688270568848
Train loss 1.2754912376403809
Train loss 1.814592719078064
2.334268528740361 0.794
Train loss 1.5687938928604126
Train loss 1.732311725616455
Train loss 1.1551512479782104
Train loss 1.2417047023773193
Train loss 0.9111664891242981
Train loss 1.3209998607635498
Train loss 1.3580964803695679
Train loss 0.9901744723320007
Train loss 0.7112072706222534
Train loss 0.7973817586898804
Train loss 1.6288416385650635
Train loss 0.7846824526786804
Train loss 1.0067877769470215
Train loss 1.4551539421081543
1.1182590082681403 0.822
Train loss 0.6152487993240356
Train loss 0.577703595161438
Train loss 0.9958917498588562
Train loss 0.8031949400901794
Train loss 1.0001251697540283

KeyboardInterrupt: 

In [17]:
model = model.to('cpu')
torch.save(model.state_dict(), 'model.pt')

run.py 内容如下：

```python
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

# Check if GPU is available
device = torch.device("cpu")

class XunFeiDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = 0
        
        return img, torch.from_numpy(np.array(label).astype(int))
    
    def __len__(self):
        return len(self.img_path)
    
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet50(False)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    
    def forward(self, img):
        out = self.resnet(img)
        return out
    
def predict(test_loader, model):
    model.eval()    
    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)


test_path = glob.glob('/work/data/birds-test-dataset/*')
test_path.sort()

test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path[:],
    transforms.Compose([
                transforms.Resize((224, 224)),
                # transforms.RandomResizedCrop(224),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])), batch_size=30, shuffle=False, num_workers=1, pin_memory=False
)
model = XunFeiNet()
model.load_state_dict(torch.load('./model/model.pt'))

test_pred = predict(test_loader, model)
test_pred = test_pred.argmax(1)
# class_names = np.array(['Asian Green Bee-Eater', 'Brown-Headed Barbet', 'Cattle Egret',
#        'Common Kingfisher', 'Common Myna', 'Common Rosefinch',
#        'Common Tailorbird', 'Coppersmith Barbet', 'Forest Wagtail',
#        'Gray Wagtail', 'Hoopoe', 'House Crow', 'Indian Grey Hornbill',
#        'Indian Peacock', 'Indian Pitta', 'Indian Roller',
#        'Jungle Babbler', 'Northern Lapwing', 'Red-Wattled Lapwing',
#        'Ruddy Shelduck', 'Rufous Treepie', 'Sarus Crane', 'White Wagtail',
#        'White-Breasted Kingfisher', 'White-Breasted Waterhen'])
# test_pred = class_names[test_pred]

pd.DataFrame({
    'imageID': [x.split('/')[-1] for x in test_path],
    'label': test_pred
}).to_csv('/work/output/result.csv', index=None)
```